# Setup

## Import Statements

In [1]:
# Import statement
import pandas as pd
import numpy as np
import os
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Import custom functions
from models2 import linreg, logreg, lda, nb, dtl, el_all, el_lin, el_rf

# Display all rows
pd.set_option('display.max_rows', None)

# Helper Functions

## Evaluation metrics for classifiers - evaluate_classifier()

In [2]:
def evaluate_classifier(clf, train_data, test_data, unseen_data):
    
    # Get the actual/predicted labels for testing and unseen data
    y_test, y_test_pred, y_unseen, y_unseen_pred,_ = clf(train_data,test_data,unseen_data)
    
    # Get metrics for test/unseen data in tuples
    accuracy = (accuracy_score(y_test, y_test_pred),accuracy_score(y_unseen, y_unseen_pred))
    precision = (precision_score(y_test, y_test_pred, zero_division=1),precision_score(y_unseen, y_unseen_pred,zero_division=1))
    recall = (recall_score(y_test, y_test_pred),recall_score(y_unseen, y_unseen_pred))
    f1 = (f1_score(y_test, y_test_pred),f1_score(y_unseen, y_unseen_pred))
    roc_auc = (roc_auc_score(y_test, y_test_pred),roc_auc_score(y_unseen, y_unseen_pred))

    # Return the list of metrics
    return [accuracy, precision, recall, f1, roc_auc]

## Get the table of metrics for all classifiers - get_metricTable()

In [3]:
def get_metricTable(c_list, train_data, test_data, unseen_data):
    
    # Get the classifier names
    c_names = [algo.__name__ for algo in c_list]
    
    # Create first column as metric and rest should be classifier names
    columns = ['metric','dataset'] + c_names
    
    # Create dataframe to hold metrics table
    results = pd.DataFrame(columns=columns)
    
    # List of metric names and their data
    metrics = ['Accuracy %', 'Precision %', 'Recall %', 'F1 %', 'ROC-AUC']
    metric_data = []
    
    # Initialize the dataframe with rows for each metric and dataset
    for metric in metrics:
        metric_data.append({'metric': metric, 'dataset': 'Test'})
        metric_data.append({'metric': metric, 'dataset': 'Unseen'})
        metric_data.append({'metric': metric, 'dataset': 'Average'}) 
    
    # Convert the list of dictionaries to a DataFrame
    results = pd.DataFrame(metric_data)
   
    
    # Evaluate each classifier and add their metrics to the dataframe
    for i, clf in enumerate(c_list):
        
        print(f"Calculating metrics for {clf.__name__}")
        
        # Get metrics for the current classifier
        metrics = evaluate_classifier(clf, train_data, test_data, unseen_data)
        
    # Write the metrics to the dataframe and calculate averages
        for j, metric in enumerate(metrics):
            test_metric = metric[0]
            unseen_metric = metric[1]
            average_metric = (test_metric + unseen_metric) / 2 
            
            # Round to 1 decimal place if ROC-AUC, rest all 2 for pct% representation
            if j == 4:
                results.at[3*j, c_names[i]] = round(test_metric,1)
                results.at[3*j+1, c_names[i]] = round(unseen_metric,1)
                results.at[3*j+2, c_names[i]] = round(average_metric, 1)
            else:
                results.at[3*j, c_names[i]] = round(test_metric*100,2)
                results.at[3*j+1, c_names[i]] = round(unseen_metric*100,2)
                results.at[3*j+2, c_names[i]] = round(average_metric*100, 2) 

    return results
       

## Read in data

In [4]:
# Read data
train_dataset = pd.read_csv("data/ml_train_data_11-12-23.csv")
test_dataset = pd.read_csv("data/ml_test_data_11-12-23.csv")
unseen_dataset = pd.read_csv("data/ml_unseen_data_11-12-23.csv")

# Data Analysis

## Get metrics

In [5]:
# Define algorithm list
algos = [linreg,logreg,lda,nb, dtl, el_all, el_lin, el_rf]

m_table = get_metricTable(algos, train_dataset, test_dataset, unseen_dataset)

m_table

Calculating metrics for linreg
Calculating metrics for logreg
Calculating metrics for lda
Calculating metrics for nb
Calculating metrics for dtl
Calculating metrics for el_all
Ensembling linreg
Ensembling logreg
Ensembling lda
Ensembling nb
Ensembling dtl
Calculating metrics for el_lin
Ensembling linreg
Ensembling lda
Ensembling dtl
Calculating metrics for el_rf


/Users/farzanmirza/Desktop/Drive/Drexel/Current/CS613/Prj/models2.py:309: RuntimeWarning: divide by zero encountered in divide
  return np.exp(-1*((X_i-mean_i)**2)/2*std_i**2)/(std_i*math.sqrt(2*math.pi))
/Users/farzanmirza/Desktop/Drive/Drexel/Current/CS613/Prj/models2.py:275: RuntimeWarning: invalid value encountered in divide
  posterior_per_observation[key] = posterior_per_observation[key] / result


,metric,dataset,linreg,logreg,lda,nb,dtl,el_all,el_lin,el_rf
0,Accuracy %,Test,66.35,50.14,65.56,54.75,49.86,54.88,66.29,50.14
1,Accuracy %,Unseen,65.80,50.24,65.77,59.15,49.75,59.10,65.17,50.25
2,Accuracy %,Average,66.08,50.19,65.66,56.95,49.81,56.99,65.73,50.19
3,Precision %,Test,64.99,50.00,63.88,52.55,49.86,52.57,62.31,100.00
4,Precision %,Unseen,63.15,50.00,63.24,55.81,49.75,55.37,61.16,100.00
5,Precision %,Average,64.07,50.00,63.56,54.18,49.81,53.97,61.74,100.00
6,Recall %,Test,70.49,99.93,71.16,95.12,100.00,97.16,82.00,0.00
7,Recall %,Unseen,75.05,99.37,74.51,85.93,100.00,91.69,82.19,0.00
8,Recall %,Average,72.77,99.65,72.83,90.53,100.00,94.43,82.10,0.00
9,F1 %,Test,67.63,66.65,67.32,67.70,66.54,68.22,70.81,0.00
